## Eksplorasi Data Transaksi

#### Import Libraries

In [36]:
import pandas as pd
import googlemaps

import numpy as np

import requests

import openpyxl

import matplotlib.pyplot as plt
import datetime

#### Load Data

In [37]:
# Load the Excel file into a pandas DataFrame
df = pd.read_excel('../data/df_distance.xlsx')
df.head()

,id,id_telp,email,tanggal,jenis_pesanan,datetime_tugas,lokasi_jemput,lat_jemput,long_jemput,lokasi_tujuan,lat_tujuan,long_tujuan,datetime_selesai,jarak,pendapatan_bersih,poin_tambahan,jenis_pembayaran,jarak_jemput_tujuan
0,1,8.212093e+10,brigezwawan72@gmail.com,5/1/2023,Ride,2023-05-01 14:01:00,"Jl. Cigadung Raya Barat No.6, Cigadung",-6.871760,107.620055,"Rancakalong RT/RW 04/04, Cigadung",-6.822729,107.844186,2023-05-01 14:11:00,1.67,8000,150.0,Dompet,70.216
1,2,8.212093e+10,brigezwawan72@gmail.com,5/1/2023,Ride,2023-05-01 12:16:00,"Jl. Sukamulya, Kopo, Bojongloa Kaler",-6.936636,107.585652,"Jl. Cibogo Atas No. 148, Sukawarna",-6.882776,107.575037,2023-05-01 12:49:00,9.03,18400,150.0,Tunai,18.526
2,3,8.212093e+10,brigezwawan72@gmail.com,5/1/2023,Ride,2023-05-01 11:50:00,Jl. Jendral Ahmad Yani Kiaracondong,-6.904833,107.647971,"196b, Gg. Pabrik Kulit Selatan No.2",-6.939039,107.589858,2023-05-01 12:15:00,9.55,19200,150.0,Tunai,9.526
3,4,8.212093e+10,brigezwawan72@gmail.com,5/1/2023,Food,2023-05-01 11:35:00,"Gg. Irit No.14, Pasirlayung, Kec. Cibeunying K...",-6.901228,107.656385,"Jl. Padasuka No.20, Pasirlayung, Kec. Cibeunyi...",-6.901107,107.653824,2023-05-01 11:48:00,0.37,7200,150.0,Tunai,0.339
4,5,8.212093e+10,brigezwawan72@gmail.com,5/1/2023,Ride,2023-05-01 10:51:00,"Jl. Cikutra No.167, Neglasari, Kec. Cibeunying...",-6.896515,107.643428,"Jl. Cihampelas No.159, Cipaganti, Kecamatan Co...",-6.893258,107.603637,2023-05-01 11:16:00,7.62,15600,150.0,Tunai,6.584


#### Data Cleaning Distance

Cek apakah fload atau bukan -> pisahkan

In [38]:
# Fungsi untuk memeriksa apakah nilai dapat diubah menjadi float atau tidak
def is_float(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

# Memeriksa kolom jarak_transaksi
invalid_jarak_transaksi = df[~df['jarak'].apply(is_float)]

# Memeriksa kolom jarak_google_maps
invalid_jarak_google_maps = df[~df['jarak_jemput_tujuan'].apply(is_float)]

In [39]:
invalid_jarak_transaksi

,id,id_telp,email,tanggal,jenis_pesanan,datetime_tugas,lokasi_jemput,lat_jemput,long_jemput,lokasi_tujuan,lat_tujuan,long_tujuan,datetime_selesai,jarak,pendapatan_bersih,poin_tambahan,jenis_pembayaran,jarak_jemput_tujuan


In [40]:
invalid_jarak_google_maps

,id,id_telp,email,tanggal,jenis_pesanan,datetime_tugas,lokasi_jemput,lat_jemput,long_jemput,lokasi_tujuan,lat_tujuan,long_tujuan,datetime_selesai,jarak,pendapatan_bersih,poin_tambahan,jenis_pembayaran,jarak_jemput_tujuan


Convert menjadi float

In [41]:
# convert kolom-kolom jarak menjadi float
df['jarak'] = df['jarak'].astype(float)
df['jarak_jemput_tujuan'] = df['jarak_jemput_tujuan'].astype(float)

In [45]:
df['jarak'].describe()

count    28947.000000
mean         3.784628
std          3.193548
min          0.000000
25%          1.670000
50%          2.880000
75%          4.870000
max         45.450000
Name: jarak, dtype: float64

#### Hitung perbedaan antara jarak asli dan jarak hasil geocoding dan distance matrix

In [47]:
# Menghitung Perbedaan Jarak
df['perbedaan_jarak'] = abs(df['jarak'] - df['jarak_jemput_tujuan'])
df.head()

,id,id_telp,email,tanggal,jenis_pesanan,datetime_tugas,lokasi_jemput,lat_jemput,long_jemput,lokasi_tujuan,lat_tujuan,long_tujuan,datetime_selesai,jarak,pendapatan_bersih,poin_tambahan,jenis_pembayaran,jarak_jemput_tujuan,perbedaan_jarak
0,1,8.212093e+10,brigezwawan72@gmail.com,5/1/2023,Ride,2023-05-01 14:01:00,"Jl. Cigadung Raya Barat No.6, Cigadung",-6.871760,107.620055,"Rancakalong RT/RW 04/04, Cigadung",-6.822729,107.844186,2023-05-01 14:11:00,1.67,8000,150.0,Dompet,70.216,68.546
1,2,8.212093e+10,brigezwawan72@gmail.com,5/1/2023,Ride,2023-05-01 12:16:00,"Jl. Sukamulya, Kopo, Bojongloa Kaler",-6.936636,107.585652,"Jl. Cibogo Atas No. 148, Sukawarna",-6.882776,107.575037,2023-05-01 12:49:00,9.03,18400,150.0,Tunai,18.526,9.496
2,3,8.212093e+10,brigezwawan72@gmail.com,5/1/2023,Ride,2023-05-01 11:50:00,Jl. Jendral Ahmad Yani Kiaracondong,-6.904833,107.647971,"196b, Gg. Pabrik Kulit Selatan No.2",-6.939039,107.589858,2023-05-01 12:15:00,9.55,19200,150.0,Tunai,9.526,0.024
3,4,8.212093e+10,brigezwawan72@gmail.com,5/1/2023,Food,2023-05-01 11:35:00,"Gg. Irit No.14, Pasirlayung, Kec. Cibeunying K...",-6.901228,107.656385,"Jl. Padasuka No.20, Pasirlayung, Kec. Cibeunyi...",-6.901107,107.653824,2023-05-01 11:48:00,0.37,7200,150.0,Tunai,0.339,0.031
4,5,8.212093e+10,brigezwawan72@gmail.com,5/1/2023,Ride,2023-05-01 10:51:00,"Jl. Cikutra No.167, Neglasari, Kec. Cibeunying...",-6.896515,107.643428,"Jl. Cihampelas No.159, Cipaganti, Kecamatan Co...",-6.893258,107.603637,2023-05-01 11:16:00,7.62,15600,150.0,Tunai,6.584,1.036


In [53]:
correct = df[df['perbedaan_jarak']<5]
correct.shape

(27437, 19)

In [54]:
correct.head()

,id,id_telp,email,tanggal,jenis_pesanan,datetime_tugas,lokasi_jemput,lat_jemput,long_jemput,lokasi_tujuan,lat_tujuan,long_tujuan,datetime_selesai,jarak,pendapatan_bersih,poin_tambahan,jenis_pembayaran,jarak_jemput_tujuan,perbedaan_jarak
2,3,8.212093e+10,brigezwawan72@gmail.com,5/1/2023,Ride,2023-05-01 11:50:00,Jl. Jendral Ahmad Yani Kiaracondong,-6.904833,107.647971,"196b, Gg. Pabrik Kulit Selatan No.2",-6.939039,107.589858,2023-05-01 12:15:00,9.55,19200,150.0,Tunai,9.526,0.024
3,4,8.212093e+10,brigezwawan72@gmail.com,5/1/2023,Food,2023-05-01 11:35:00,"Gg. Irit No.14, Pasirlayung, Kec. Cibeunying K...",-6.901228,107.656385,"Jl. Padasuka No.20, Pasirlayung, Kec. Cibeunyi...",-6.901107,107.653824,2023-05-01 11:48:00,0.37,7200,150.0,Tunai,0.339,0.031
4,5,8.212093e+10,brigezwawan72@gmail.com,5/1/2023,Ride,2023-05-01 10:51:00,"Jl. Cikutra No.167, Neglasari, Kec. Cibeunying...",-6.896515,107.643428,"Jl. Cihampelas No.159, Cipaganti, Kecamatan Co...",-6.893258,107.603637,2023-05-01 11:16:00,7.62,15600,150.0,Tunai,6.584,1.036
5,6,8.212093e+10,brigezwawan72@gmail.com,5/1/2023,Ride,2023-05-01 10:14:00,"Jl. Gegerkalong Hilir No.8, RW.01, Gegerkalong...",-6.870245,107.593153,"Jl. Jiwanaya No.02, RT.01/RW.04, Cibeunying, K...",-6.879492,107.631440,2023-05-01 10:39:00,7.80,16000,150.0,Dompet,8.390,0.590
6,7,8.212093e+10,brigezwawan72@gmail.com,5/1/2023,Ride,2023-05-01 09:05:00,"Cicaheum, Kec. Kiaracondong, Kota Bandung, Jaw...",-6.903948,107.652282,"Jl. Puri Dago Mas Utara 1 No.33, Antapani Teng...",-6.922764,107.665858,2023-05-01 09:17:00,3.90,8000,150.0,Dompet,3.817,0.083


In [58]:
# Load the Excel file into a pandas DataFrame
check = pd.read_excel('../data/after_check.xlsx')
check.shape

/home/peminumyakult/anaconda3/envs/suapp/lib/python3.11/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


(46, 19)

In [60]:
final_df = pd.concat([correct, check], ignore_index=True)
final_df.shape

(27483, 19)